# Lab | Handling Data Imbalance in Classification Models

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
churnData = pd.read_csv('Customer-Churn.csv')

In [3]:
churnData

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [4]:

print(churnData.dtypes)

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


In [6]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [7]:
churnData.isna().sum() 

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [8]:
churnData = churnData.dropna()

In [10]:
features = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
target = 'Churn'


In [21]:
X_train, X_test, y_train, y_test = train_test_split(churnData[features], churnData[target], test_size=0.3, random_state=42)


In [22]:
# Scaling using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
# Fitting a logistic regression model on the training data
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

LogisticRegression()

In [24]:
# Check the accuracy on the test data.
y_pred = logreg.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.781042654028436


In [25]:
# Check for the imbalance.

In [26]:
churnData[target].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

In [27]:
# The data is imbalanced.

Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.

In [40]:
from imblearn.under_sampling import TomekLinks
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report

In [34]:
tl = TomekLinks('majority')

X_tl, y_tl = tl.fit_resample(np.array(X_train), y_train)

y_tl.value_counts()

No     3234
Yes    1308
Name: Churn, dtype: int64

In [35]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

X_sm, y_sm = smote.fit_resample(np.array(X_train), y_train)
y_sm.value_counts()

Yes    3614
No     3614
Name: Churn, dtype: int64

In [37]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_resample(np.array(X_train), y_train)
print('1st TkLinks:')
print(y_tl.value_counts())

smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X_tl, y_tl)
print('SMOTE:')
print(y_sm.value_counts())

tl = TomekLinks('all')
X_tl, y_tl = tl.fit_resample(X_sm, y_sm)
print('2nd TkLinks:')
print(y_tl.value_counts())

1st TkLinks:
No     3234
Yes    1308
Name: Churn, dtype: int64
SMOTE:
Yes    3234
No     3234
Name: Churn, dtype: int64
2nd TkLinks:
Yes    3056
No     3056
Name: Churn, dtype: int64


/Users/usuari/opt/anaconda3/lib/python3.9/site-packages/imblearn/utils/_validation.py:635: FutureWarning: Pass sampling_strategy=all as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


In [43]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(n_jobs=7)

model.fit(X_tl,y_tl)

preds = model.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          No       0.89      0.70      0.78      1549
         Yes       0.48      0.77      0.59       561

    accuracy                           0.72      2110
   macro avg       0.69      0.73      0.69      2110
weighted avg       0.78      0.72      0.73      2110



/Users/usuari/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [44]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(n_jobs=7)

model.fit(X_sm,y_sm)

preds = model.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          No       0.89      0.70      0.79      1549
         Yes       0.48      0.77      0.59       561

    accuracy                           0.72      2110
   macro avg       0.69      0.74      0.69      2110
weighted avg       0.79      0.72      0.74      2110



/Users/usuari/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [45]:
# The accuracy hasn't really changed that much between the upsamplig and downsampling strategies 
